# Synthetic Data Vault (SDV)
In this notebook, we will use Synthetic Data Vault (SDV) to create sequential synthetic data. The foundation of this notebook is the [tutorial from SDV](https://colab.research.google.com/drive/1cT4-jFK2Bxe93QudC_CwHq_yVCcNcxal?usp=sharing) and the notebook on DataSynthesizer.

Both served as inspiration in creating synthetic data with SDV in this manner.

In [1]:
# import relevant modules
import sdv
import pandas as pd
import numpy as np
import os

# Load data

In [2]:
data = pd.read_csv('CMAPSS/train_FD001.txt', sep=" ", header=None)

# drop last two columns with N/A values
data = data.iloc[:, :-2]

# rename columns according to readme.txt
col_names = ["unit-nr", "timecycle", "ops-set1", "ops-set2", "ops-set3"]
for i in range(1,22):
    col_names.append(f"sens-{i}")
data.columns = col_names
data.to_csv('CMAPSS/train_FD001_pre.csv', index=False)

data_length = len(data)

# display data
# data
# data.columns

# Create synthetic data

In [3]:
# Create metadata
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

# load data
metadata.detect_from_dataframe(data=data)

# set unit-nr as sequence key, timecycle as sequence index
metadata.update_column(
    column_name='unit-nr',
    sdtype='id')
metadata.set_sequence_key('unit-nr')
metadata.set_sequence_index('timecycle')

# show metadata
# metadata

In [4]:
"""
In this example, we do not need context_columns.
As all columns have an alternating value, none
remains constant or context-dependent.

"""
from sdv.sequential import PARSynthesizer

# create synthesizer
synthesizer = PARSynthesizer(
    metadata,
    epochs=100,
    verbose=True
    )

synthesizer.fit(data)

Epoch 100 | Loss -0.44507795572280884: 100%|██| 100/100 [02:05<00:00,  1.25s/it]


In [5]:
# find amount of unique unit-nrs
n_unitnrs = len(data['unit-nr'].unique())

# create synthetic data with that same amount of unit-nrs
synthetic_data = synthesizer.sample(num_sequences=n_unitnrs)
synthetic_data

# export SyntData to folder
synthetic_data.to_csv('./CMAPSS/Synthetic/SDV_FD001.csv')

100%|█████████████████████████████████████████| 100/100 [01:32<00:00,  1.08it/s]


# Analysis
Cool! We've created our first synthetic data, now we wish to analyse this synthetic data. More specifically, we would like to compare the output and distributions of our synthetic data compared to the original data.

In [6]:
data_description = data.describe()
data_description

,unit-nr,timecycle,ops-set1,ops-set2,ops-set3,sens-1,sens-2,sens-3,sens-4,sens-5,...,sens-12,sens-13,sens-14,sens-15,sens-16,sens-17,sens-18,sens-19,sens-20,sens-21
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.0,20631.00,20631.000000,20631.000000,20631.000000,2.063100e+04,...,20631.000000,20631.000000,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000
mean,51.506568,108.807862,-0.000009,0.000002,100.0,518.67,642.680934,1590.523119,1408.933782,1.462000e+01,...,521.413470,2388.096152,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705
std,29.227633,68.880990,0.002187,0.000293,0.0,0.00,0.500053,6.131150,9.000605,1.776400e-15,...,0.737553,0.071919,19.076176,0.037505,1.387812e-17,1.548763,0.0,0.0,0.180746,0.108251
min,1.000000,1.000000,-0.008700,-0.000600,100.0,518.67,641.210000,1571.040000,1382.250000,1.462000e+01,...,518.690000,2387.880000,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200
25%,26.000000,52.000000,-0.001500,-0.000200,100.0,518.67,642.325000,1586.260000,1402.360000,1.462000e+01,...,520.960000,2388.040000,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800
50%,52.000000,104.000000,0.000000,0.000000,100.0,518.67,642.640000,1590.100000,1408.040000,1.462000e+01,...,521.480000,2388.090000,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900
75%,77.000000,156.000000,0.001500,0.000300,100.0,518.67,643.000000,1594.380000,1414.555000,1.462000e+01,...,521.950000,2388.140000,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800
max,100.000000,362.000000,0.008700,0.000600,100.0,518.67,644.530000,1616.910000,1441.490000,1.462000e+01,...,523.380000,2388.560000,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400


In [7]:
syn_description = synthetic_data.describe()
syn_description

,unit-nr,timecycle,ops-set1,ops-set2,ops-set3,sens-1,sens-2,sens-3,sens-4,sens-5,...,sens-12,sens-13,sens-14,sens-15,sens-16,sens-17,sens-18,sens-19,sens-20,sens-21
count,23687.000000,23687.000000,23687.000000,23687.000000,23687.0,23687.00,23687.000000,23687.000000,23687.000000,2.368700e+04,...,23687.000000,23687.000000,23687.000000,23687.000000,2.368700e+04,23687.000000,23687.0,23687.0,23687.000000,23687.000000
mean,48598.832693,133.089374,-0.000056,0.000005,100.0,518.67,642.733602,1591.369250,1410.287548,1.462000e+01,...,521.333383,2388.103602,8145.846806,8.447381,3.000000e-02,393.395449,2388.0,100.0,38.791790,23.277042
std,26249.294173,89.670899,0.001056,0.000152,0.0,0.00,0.273815,3.243974,5.392316,3.552789e-15,...,0.429735,0.037836,13.007180,0.020914,1.734760e-17,0.918140,0.0,0.0,0.103855,0.060912
min,476.000000,1.000000,-0.004213,-0.000600,100.0,518.67,641.706398,1578.471903,1390.592358,1.462000e+01,...,519.617555,2387.959022,8099.940000,8.355087,3.000000e-02,390.000000,2388.0,100.0,38.397895,23.021687
25%,28574.000000,60.000000,-0.000738,-0.000094,100.0,518.67,642.561008,1589.314865,1406.742171,1.462000e+01,...,521.045760,2388.079819,8137.454104,8.433732,3.000000e-02,393.000000,2388.0,100.0,38.722322,23.236924
50%,48479.000000,119.000000,-0.000009,0.000002,100.0,518.67,642.714209,1591.186161,1410.121169,1.462000e+01,...,521.350893,2388.101983,8145.204948,8.446768,3.000000e-02,393.000000,2388.0,100.0,38.794869,23.280143
75%,68964.000000,194.000000,0.000618,0.000104,100.0,518.67,642.914956,1593.532483,1413.907929,1.462000e+01,...,521.611186,2388.128643,8154.564755,8.461495,3.000000e-02,394.000000,2388.0,100.0,38.859839,23.316176
max,99711.000000,362.000000,0.004178,0.000600,100.0,518.67,643.810067,1606.807363,1432.680089,1.462000e+01,...,522.969677,2388.277698,8207.248460,8.536340,3.000000e-02,397.000000,2388.0,100.0,39.207847,23.512199


What directly stands out is that the unit-nrs are not ordered from 1-100. However, this is due to the very nature of SDV: it will never create instances that overlap with the original data [source](https://colab.research.google.com/drive/1YLk2uwn8yrSRPy0soEeJwu8Hdk_tGTlE#scrollTo=IjvfTOMVKQot). Therefore, this behaviour is to be expected.

The descriptions look rather similar, most means and standarddeviations are close to the description of the original data. Therefore it would be interesting to test the SDV data on a ML task.

## SDV evaluation

In [8]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    data,
    synthetic_data,
    metadata
)

# with 128 epochs:
"""Overall Quality Score: 81.07%

Properties:
- Column Shapes: 82.68%
- Column Pair Trends: 79.45%
"""

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|█| 325/325 [00:00<00:00, 453.22it/s]

Overall Quality Score: 81.28%

Properties:
- Column Shapes: 83.11%
- Column Pair Trends: 79.45%


'Overall Quality Score: 81.07%\n\nProperties:\n- Column Shapes: 82.68%\n- Column Pair Trends: 79.45%\n'

# Testing properties